In [1]:
# 05/31/22

import os, torch, torch.nn as nn, torch.utils as utils, tqdm, numpy as np, random

In [2]:
folder = "../tempdata/new_split/train/fingerprint"

print(f"Num molecules to rank against: {len(os.listdir(folder))}")

Num molecules to rank against: 9794


In [3]:
mols = []
for f in tqdm.tqdm(os.listdir(folder)):
    mols.append(np.load(os.path.join(folder, f)))
print(f"Done Loading")
ar = np.stack(mols)

100%|██████████| 9794/9794 [00:02<00:00, 4164.17it/s]


Done Loading


In [4]:
ar.shape

(9794, 6144)

In [5]:
from numpy import dot
from numpy.linalg import norm
query = np.load("../tempdata/new_split/val/fingerprint/mol_6.npy").reshape((1,-1))

def cossim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim
# get dot product of query with everything in library [library size, 1]
dots = ar @ np.transpose(query)
# get norm of query
norm_q = norm(query)
# get all norms in the array [library size, 1]
norm_ar = norm(ar, axis=1).reshape((-1, 1))

In [15]:
print(dots.shape, norm_q, norm_ar.shape)

(9794, 1) 8.54400374531753 (9794, 1)


In [13]:
cossims = dots / (norm_ar * norm_q)

In [14]:
# sanity check: make sure that altering fp would change the ranking

q2 = np.copy(query)
ct_flip = 500
# generate 500 random indices
idxs = np.random.choice(range(6144), ct_flip, replace=False)
# for each of those indices in the query, invert them : 1->0 0->1
q2[0,idxs] = 1- q2[0,idxs]
# get the cosine similarity between the original query and the query 
# with inverted values
thresh = cossim(q2.flatten(), query.flatten())
print(f"Thresh: {thresh}")

# the number of cosine similarities greater than this threhshold
ct_greater = np.sum(cossims >= thresh)
print(f"Ct greater: {ct_greater}")

Thresh: 0.3354233446943441
Ct greater: 15


In [28]:
for i in range(10):
    rd_idx = random.randint(0, 6143)
    query[rd_idx] = not query[rd_idx]
print(np.cos)

IndexError: index 5475 is out of bounds for axis 0 with size 1